Using abcTau to fit ACFs for trials (Figure 2 from the paper). 

Three options to do that:
* use abcTau package for both ACF and fitting
* use ACF calculated before using acf function
* use ACF calculated before using iSTTC concat function

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns 

import pickle
import re
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import stats

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
sys.path.append('C:\\Users\\ipochino\\AppData\\Local\\anaconda3\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
import abcTau

from isttc.scripts.cfg_global import project_folder_path

In [2]:
dataset_folder = project_folder_path + 'synthetic_dataset\\'
results_folder = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau\\'

results_folder_abctau = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau_abctau\\'
datasave_path = results_folder_abctau + 'all_abctau\\'

### Load data (spike trains and calculated acf's)

In [3]:
df_all = pd.read_pickle(results_folder + "summary_tau_all_long_df_all_units.pkl")

In [ ]:
with open(dataset_folder + 'trials40_binned.pkl','rb') as f:
    data_binned = pickle.load(f)

trial_dict_binned = data_binned['trial_dict']
alphas_binned  = data_binned['alphas']
fr_values_binned  = data_binned['fr_values']
taus_ms_binned  = data_binned['tau_ms']
n_trials_binned  = data_binned['n_trials']
trial_lens_binned  = data_binned['trial_lens']

print(f'n spike trains {len(trial_dict_binned)}, trial_lens {trial_lens_binned[0]} ms')

In [ ]:
with open(results_folder + 'tau_pearsonr_trial_50ms_20lags_dict.pkl', "rb") as f:
    pearsonr_trial_avg_dict = pickle.load(f)

with open(results_folder + 'tau_isttc_trial_concat_50ms_20lags_dict.pkl', "rb") as f:
    sttc_trial_concat_dict = pickle.load(f)

print(f'len pearsonr_trial_avg_dict {len(pearsonr_trial_avg_dict)}')
print(f'len sttc_trial_concat_dict {len(sttc_trial_concat_dict)}')

In [4]:
lv_df = pd.read_pickle(results_folder + 'lv_df.pkl')
print(f'Lv loaded for n spike trains {len(lv_df)}')

lv_df

Lv loaded for n spike trains 100000


,unit_id,alpha,fr,tau_ms,lv
0,0,0.851099,7.741821,150.923515,1.307775
1,1,0.858098,4.394396,147.947233,1.345006
2,2,0.528805,8.587393,267.002290,1.068416
3,3,0.300692,6.976707,194.403797,1.055631
4,4,0.691458,0.950832,239.986489,1.357205
...,...,...,...,...,...
99995,99995,0.244627,2.672839,125.243251,1.178739
99996,99996,0.596987,5.754497,94.481295,1.353800
99997,99997,0.258915,5.604441,174.189672,1.064661
99998,99998,0.187620,1.219057,76.565797,1.306500


### Summary df

In [5]:
directory = Path(datasave_path)
files = [file for file in directory.iterdir() if file.is_file()]
print(f'N files {len(files)}')

records = []
for file in files:
    # print(file.name)  
    # the last iteration is encoded in the name
    ind = file.name.find('steps') 
    final_step = int(file.name[ind+5] + file.name[ind+6])
    # het results 
    abc_results = np.load(file, allow_pickle=True)
    theta_accepted = abc_results[final_step-1]['theta accepted']
    tau = theta_accepted[0]

    match_unit_id = re.search(r"spike_train_(\d+)_", file.name)
    unit_id = int(match_unit_id.group(1))
    records.append({
        'unit_id': unit_id,
        'tau_mean': np.mean(tau),
        'tau_median': np.median(tau),
        'method': 'abctau_full',
    })

tau_df = pd.DataFrame(records)

N files 318


In [ ]:
tau_df

In [6]:
tau_df_merged = tau_df.merge(lv_df, how='left', on='unit_id')
tau_df_merged['tau_diff_abs'] = np.abs(tau_df_merged['tau_mean'] - tau_df_merged['tau_ms'])
tau_df_merged['tau_diff_rel'] = tau_df_merged['tau_diff_abs'] / tau_df_merged['tau_ms'] * 100
tau_df_merged.sort_values(by='tau_diff_rel')

tau_df_merged.rename(columns={'tau_ms': 'tau_ms_true'}, inplace=True)
tau_df_merged.rename(columns={'tau_mean': 'tau_ms'}, inplace=True)
tau_df_merged.drop('tau_median', axis=1, inplace=True)

tau_df_merged

,unit_id,tau_ms,method,alpha,fr,tau_ms_true,lv,tau_diff_abs,tau_diff_rel
0,149,387.985142,abctau_full,0.777130,0.225905,200.387112,1.407389,187.598030,93.617812
1,148,128.579548,abctau_full,0.883786,4.394725,222.802384,1.369385,94.222836,42.289869
2,147,37.244705,abctau_full,0.844883,5.211517,222.150701,1.342155,184.905996,83.234487
3,146,52.888489,abctau_full,0.749484,3.322374,151.730964,1.410801,98.842474,65.143246
4,145,18.560043,abctau_full,0.591272,2.677087,202.714438,1.337291,184.154394,90.844242
...,...,...,...,...,...,...,...,...,...
313,181,28.556237,abctau_full,0.582557,8.935532,98.477833,1.233783,69.921596,71.002370
314,180,23.886875,abctau_full,0.756569,8.909017,297.282467,1.175163,273.395592,91.964923
315,179,11.355139,abctau_full,0.488050,7.488594,195.635682,1.128110,184.280543,94.195773
316,178,6.889937,abctau_full,0.214350,8.917856,282.249144,1.036031,275.359207,97.558917


In [7]:
df_all_subset = df_all[df_all['unit_id'].isin(tau_df['unit_id'].values)][['unit_id', 'method', 'tau_ms', 'fr', 'alpha', 'tau_ms_true',
       'tau_diff_abs', 'tau_diff_rel', 'lv']].copy()
df_all_subset.reset_index(drop=True, inplace=True)
df_all_subset

,unit_id,method,tau_ms,fr,alpha,tau_ms_true,tau_diff_abs,tau_diff_rel,lv
0,0,acf_full,1.108950e+02,7.741821,0.851099,150.923515,4.002854e+01,2.652240e+01,1.307775
1,1,acf_full,1.284105e+02,4.394396,0.858098,147.947233,1.953672e+01,1.320520e+01,1.345006
2,2,acf_full,2.613726e+02,8.587393,0.528805,267.002290,5.629722e+00,2.108492e+00,1.068416
3,3,acf_full,1.975105e+02,6.976707,0.300692,194.403797,3.106670e+00,1.598050e+00,1.055631
4,4,acf_full,1.858982e+02,0.950832,0.691458,239.986489,5.408831e+01,2.253807e+01,1.357205
...,...,...,...,...,...,...,...,...,...
1267,1131,sttc_trial_concat,3.536355e+06,5.169429,0.233285,175.423723,3.536180e+06,2.015793e+06,1.076749
1268,1132,sttc_trial_concat,1.275869e+02,0.961039,0.537046,67.094931,6.049194e+01,9.015873e+01,1.753471
1269,1133,sttc_trial_concat,4.342650e+01,7.281155,0.537366,109.054189,6.562769e+01,6.017897e+01,1.261342
1270,1134,sttc_trial_concat,1.776440e+00,9.766874,0.111308,248.778167,2.470017e+02,9.928593e+01,1.002391


In [8]:
df_5_methods = pd.concat([tau_df_merged, df_all_subset])
df_5_methods.reset_index(drop=True, inplace=True)

df_5_methods

,unit_id,tau_ms,method,alpha,fr,tau_ms_true,lv,tau_diff_abs,tau_diff_rel
0,149,3.879851e+02,abctau_full,0.777130,0.225905,200.387112,1.407389,1.875980e+02,9.361781e+01
1,148,1.285795e+02,abctau_full,0.883786,4.394725,222.802384,1.369385,9.422284e+01,4.228987e+01
2,147,3.724471e+01,abctau_full,0.844883,5.211517,222.150701,1.342155,1.849060e+02,8.323449e+01
3,146,5.288849e+01,abctau_full,0.749484,3.322374,151.730964,1.410801,9.884247e+01,6.514325e+01
4,145,1.856004e+01,abctau_full,0.591272,2.677087,202.714438,1.337291,1.841544e+02,9.084424e+01
...,...,...,...,...,...,...,...,...,...
1585,1131,3.536355e+06,sttc_trial_concat,0.233285,5.169429,175.423723,1.076749,3.536180e+06,2.015793e+06
1586,1132,1.275869e+02,sttc_trial_concat,0.537046,0.961039,67.094931,1.753471,6.049194e+01,9.015873e+01
1587,1133,4.342650e+01,sttc_trial_concat,0.537366,7.281155,109.054189,1.261342,6.562769e+01,6.017897e+01
1588,1134,1.776440e+00,sttc_trial_concat,0.111308,9.766874,248.778167,1.002391,2.470017e+02,9.928593e+01


In [9]:
df_5_methods.to_pickle(results_folder + 'summary_tau_all_long_df_all_units_with_abctau.pkl')